In [ ]:
import pandas as pd
import numpy as np
import numpy.linalg as la
from urllib.parse import unquote

In [ ]:
df = pd.read_csv('links.tsv', sep='\t', header=None, skiprows=12, encoding='utf-8')

In [ ]:
df.head()

## Note the encoded urls of the initial dataset. The following decodes it into a more readable format; these are still valid wikipedia endpoints.

In [ ]:
df.iloc[:,0] = df.iloc[:,0].apply(unquote)
df.iloc[:, 1] = df.iloc[:,1].apply(unquote)

In [ ]:
df

## We made a design decision to remove all links that link back to themselves. This was because of our implementation, and noting we don't lose too many entries on dropping these

In [ ]:
cycles = df[df.iloc[:,0] == df.iloc[:,1]]
cycles

In [ ]:
df_no_cycs = df.drop(cycles.index)
df_no_cycs

## Writes our decoded dataframe with dropped self-cycles to a new .tsv. This set is what we use for our graph

In [ ]:
df_no_cycs.to_csv("decoded_links.tsv", sep="\t", index=False, header=False, line_terminator="\n", encoding="utf-8")

## The following cells are used to generate expected results of a converged PageRank algorithm, based on test cases found within our code
#### Implementation based on CS357 content: https://courses.grainger.illinois.edu/cs357/fa2020/assets/lectures/complete-slides/13-Markov-Chains.pdf

In [ ]:
def normalize_cols(M):
    '''
    Normalizes the columns of a matrix to 1
    '''
    for i in range(len(M)):
        norm = la.norm(M[:,i], ord=1)
        M[:,i] = M[:,i]/norm if norm != 0 else (1/len(M))
    return M

In [ ]:
def G_mat(M, alpha, n):
    '''
    Produces the G matrix based on the size of our M and desired alpha
    '''
    ones = np.ones((n,n))
    G = alpha*M+(1-alpha)*(1/n)*ones
    return G

In [ ]:
def page_rank(M, alpha=0.85):
    '''
    The power iteration to determine the PageRank of a page
    '''
    n = M.shape[0]
    G = G_mat(M, alpha, n)
    x = np.random.rand(n)
    x = x/la.norm(x, ord=1)
    
    diff = 1
    while diff > 1e-7:
        xprev = x
        x = G@x
        diff = la.norm(x - xprev,2)
    return x

In [ ]:
def sort_results(vertices, x):
    '''
    Sorts the resulting vector, x, and the corresponding vertices
    '''
    indices = x.argsort()[::-1]
    return (x[indices], [verts[i] for i in indices])

## This matrix is the representation of our connected_graph.tsv

In [ ]:
verts = ["A", "B", "C", "D", "E", "F", "G"]
M = np.array([
    [0, 0, 0, 0, 1, 1, 0],
    [0, 0, 1, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0, 1],
    [1, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 1, 0, 0],
    [0, 0, 0, 1, 1, 0, 0],
], dtype='float64')
M = normalize_cols(M)
x = page_rank(M)
results = sort_results(verts, x)

In [ ]:
results

## This matrix is the representation of our disconnected_graph.tsv

In [ ]:
verts = ["A", "B", "C", "D", "E", "F", "G", "H"]
M = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 1, 0],
], dtype='float64')
M = normalize_cols(M)
x = page_rank(M)
results = sort_results(verts, x)

In [ ]:
results

## This matrix is the representation of our complex_graph.tsv

In [ ]:
verts = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L"]
M = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
], dtype='float64')
M = normalize_cols(M)
x = page_rank(M)
results = sort_results(verts, x)

In [ ]:
results